In [9]:
import nltk
nltk.download('stopwords')
import re
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import logging
import warnings

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kirtikharb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
from nltk.corpus import stopwords
stop_words = pd.read_csv('/Users/kirtikharb/Downloads/en.txt')
#can extend stopwords using the below command
#stop_words.extend(['from','subject','re','edu','use'])

In [36]:
df = pd.read_csv('/Users/kirtikharb/insta_dead_repo/insta-dead-article/data/posts-formatted-for-topicmodelling.csv', encoding='latin-1')

In [37]:
df.head()

Unnamed: 0  year  username  \
0           1  2011  10171087   
1           2  2011  10031678   
2           3  2011  10171087   
3           4  2011  10171087   
4           5  2011  10321446   

                                                text  
0  I brought Trevor to work!!! Haha #humanbone #t...  
1  My first #humanbone ! Thanx @USERNAME_HERE #go...  
2  My new torso came in today! I love it!! #human...  
3  Just got 2 femurs and a hand!!!! Yaaaay! #huma...  
4                          Dead hooker mistle toe...

In [38]:
#tokenize words and clean-up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True)) #deacc = True removes punctuation

data_words = list(sent_to_words(df.text))
#print(data_words)

In [39]:
#Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words,min_count=5,threshold=100) #higher threshold fewer phrases
trigram = gensim.models.Phrases(bigram[data_words],threshold=10)

In [40]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [41]:
#this will combine trigrams into 3 words with underscores so "pls fix ASAP" turns into "pls_fix_ASAP" if those 3 words occur with enough frequency together
print(trigram_mod[bigram_mod[data_words]])

In [42]:
#function to remove stop words
def remove_stopwords(texts):
    return[[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

#function to make bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

#function to make trigrams
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [22]:
data_words_nostops = remove_stopwords(data_words)

In [23]:
data_words_trigrams = make_trigrams(data_words_nostops)

In [43]:
#print(data_words_trigrams)

In [44]:
#create Dictionary
id2word = corpora.Dictionary(data_words_trigrams)

#create Corpus
texts = data_words_trigrams

#term document frequency
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(1, 1), (3, 1), (7, 1), (8, 1), (9, 1)], [(1, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2)], [(1, 1), (3, 1), (12, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [(23, 1), (24, 1), (25, 1), (26, 1)], [(27, 1), (28, 1), (29, 1), (30, 1), (31, 1)], [(1, 1), (3, 1), (12, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)], [(15, 1), (16, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(11, 1), (14, 1), (43, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1)], [(3, 1), (53, 1), (54, 1), (55, 1)], [(3, 1), (56, 1), (57, 1), (58, 1)], [(3, 1), (59, 1), (60, 1)], [(3, 1), (61, 1), (62, 1)], [(63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)], [(69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1)], [(9, 1), (15, 1), (80, 1)], [(81, 1)], [(47, 1), (82, 1), (83, 1), (84, 1)], [(3, 1), (85, 1), (86, 1), (87,

In [45]:
#human readable format of corpus (term-frequency)
[[(id2word[id],freq) for id, freq in cp] for cp in corpus]

[[('brought', 1),
  ('goldenlotus', 1),
  ('haha', 1),
  ('humanbone', 1),
  ('tattoos', 1),
  ('trevor', 1),
  ('work', 1)],
 [('goldenlotus', 1),
  ('humanbone', 1),
  ('first', 1),
  ('thanx', 1),
  ('username_here', 1)],
 [('goldenlotus', 1),
  ('came', 1),
  ('human', 1),
  ('humanremains', 1),
  ('love', 1),
  ('new', 1),
  ('realbone', 1),
  ('skeleton', 1),
  ('today', 1),
  ('torso', 2)],
 [('goldenlotus', 1),
  ('humanbone', 1),
  ('humanremains', 1),
  ('femurs', 1),
  ('got', 1),
  ('hand', 1),
  ('yaaaay', 1)],
 [('dead', 1), ('hooker', 1), ('mistle', 1), ('toe', 1)],
 [('bought', 1), ('horn', 1), ('human_femur', 1), ('mas', 1), ('tibet', 1)],
 [('goldenlotus', 1),
  ('humanbone', 1),
  ('humanremains', 1),
  ('art', 1),
  ('boneart', 1),
  ('complete', 1),
  ('door', 1),
  ('feel_like', 1),
  ('front', 1),
  ('hope', 1),
  ('like', 1),
  ('named', 1),
  ('satan', 1),
  ('working', 1)],
 [('realbone', 1),
  ('skeleton', 1),
  ('bone', 1),
  ('cat', 1),
  ('lostplace', 1),


In [46]:
#Build LDA model
#chunksize = number of docuemtns to be used in each training chunk
#update_everey = how often the model parameters should be updated
#passes = total number of training passes
lda = gensim.models.ldamodel.LdaModel(corpus, num_topics=25)

In [29]:
pprint(lda.print_topics())
doc_lda = lda[corpus]

[(12,
  '0.064*"340" + 0.054*"47" + 0.029*"72" + 0.021*"11" + 0.018*"147" + '
  '0.016*"344" + 0.015*"297" + 0.014*"607" + 0.013*"163" + 0.012*"16"'),
 (13,
  '0.070*"47" + 0.034*"1553" + 0.031*"340" + 0.016*"716" + 0.013*"8105" + '
  '0.012*"779" + 0.010*"226" + 0.009*"5355" + 0.009*"1549" + 0.009*"1138"'),
 (22,
  '0.058*"47" + 0.040*"340" + 0.021*"607" + 0.020*"147" + 0.017*"1053" + '
  '0.015*"4020" + 0.015*"7259" + 0.014*"2086" + 0.014*"224" + 0.014*"402"'),
 (10,
  '0.076*"47" + 0.058*"340" + 0.018*"147" + 0.016*"1217" + 0.013*"11" + '
  '0.009*"2722" + 0.008*"32" + 0.008*"5582" + 0.008*"2695" + 0.008*"5019"'),
 (16,
  '0.031*"8079" + 0.021*"735" + 0.020*"4631" + 0.018*"3169" + 0.017*"1993" + '
  '0.016*"1014" + 0.016*"4135" + 0.013*"5571" + 0.012*"1738" + 0.011*"6753"'),
 (20,
  '0.029*"340" + 0.022*"72" + 0.021*"43" + 0.021*"157" + 0.019*"47" + '
  '0.014*"1146" + 0.013*"2394" + 0.013*"7124" + 0.011*"6763" + 0.011*"738"'),
 (14,
  '0.036*"47" + 0.024*"533" + 0.019*"340" + 0.018

In [30]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

/Users/kirtikharb/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.238099  0.033263       1        1  12.696782
5      0.227754 -0.089086       2        1  10.075635
18     0.144319  0.047904       3        1   8.010303
17     0.182351 -0.171243       4        1   7.013123
23     0.121404  0.129421       5        1   6.365794
4      0.174669 -0.105436       6        1   5.699591
10     0.133106  0.016191       7        1   5.683066
12     0.107018  0.073041       8        1   5.112481
7      0.099937  0.064389       9        1   4.914178
24     0.038026 -0.048424      10        1   4.699200
22     0.072307  0.126249      11        1   4.109484
13     0.036917 -0.044221      12        1   3.278732
20    -0.017277  0.078024      13        1   3.118800
9     -0.090600  0.052030      14        1   2.313514
1     -0.106796 -0.023703      15        1   2.073336
2     -0.046820  0.023147      16        1   1.988100
19    -0.116925 -0.026464      17        1   1.842771
14    -0.081299 -0.059404      18        1   1.837563
15    -0.108846  0.016627      19        1   1.745091
6     -0.096512 -0.004189      20        1   1.724454
16    -0.174506 -0.010840      21        1   1.260115
21    -0.164643 -0.033757      22        1   1.210495
0     -0.186594  0.024027      23        1   1.151675
3     -0.167242 -0.044655      24        1   1.150711
11    -0.217846 -0.022892      25        1   0.925003, topic_info=     Category          Freq                         Term         Total  \
4380  Default    907.000000                  macabre_art    907.000000   
548   Default   1279.000000                      drawing   1279.000000   
1144  Default   1709.000000                       tattoo   1709.000000   
47    Default  15804.000000                        skull  15804.000000   
1718  Default    701.000000                          ink    701.000000   
9     Default   2473.000000                username_here   2473.000000   
2714  Default    881.000000                  skulltattoo    881.000000   
32    Default   2412.000000                          art   2412.000000   
1969  Default    305.000000                          wip    305.000000   
1513  Default    772.000000                       sketch    772.000000   
340   Default  13960.000000                   humanskull  13960.000000   
1553  Default    346.000000                       carved    346.000000   
779   Default    661.000000                blackandwhite    661.000000   
6026  Default    252.000000                     instaart    252.000000   
5528  Default    425.000000                      darkart    425.000000   
72    Default   2151.000000                        bones   2151.000000   
6561  Default    312.000000                 blackandgrey    312.000000   
2930  Default    229.000000                      pendant    229.000000   
105   Default    640.000000  username_here_username_here    640.000000   
621   Default    503.000000                      artwork    503.000000   
5514  Default    266.000000                    blackwork    266.000000   
8492  Default    268.000000                   sketchbook    268.000000   
470   Default    294.000000                         info    294.000000   
157   Default    882.000000                      anatomy    882.000000   
607   Default   1022.000000                     oddities   1022.000000   
3118  Default    308.000000                 skulldrawing    308.000000   
2051  Default    566.000000                     painting    566.000000   
297   Default    553.000000                      medical    553.000000   
4064  Default    650.000000                 illustration    650.000000   
3107  Default    400.000000                   art_artist    400.000000   
...       ...           ...                          ...           ...   
453   Topic25      6.225003            anyone_interested      7.939877   
2959  Topic25      5.330353                       dremel 

In [31]:
#calculate model perplexity
#a measure of how good the model is, low the score the better
print('\nPerplexity:',lda.log_perplexity(corpus))


Perplexity: -16.460636558346085


In [32]:
#calculate model coherence score
coherence_model_lda = CoherenceModel(model=lda,texts=data_words_trigrams,dictionary=id2word,coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score:',coherence_lda)


Coherence Score: 0.4422698324456151
